In [1]:
import requests,re,time,numpy as np,pickle,asyncio,concurrent.futures
# Make this notebook wiiiiiiiiide
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# Setup relevant variables
ZIP = '60615'
pickupDate = '11/3/2017'
dropoffDate = '11/10/2017'
TIME = '5:00 PM'
RADIUS = '45'

In [11]:
%run core_functions_debug.ipynb

In [4]:
results = runparser(threads=10)
print(results)

001 OK | 1 ('enterprise', 'E11586', 'Gg Hyde Park') ('0.95', '41.794955', '-87.587695')
009 ND | 9 ('avis', 'CHIX01', 'Chicago Union Train Station') ('5.71', '41.878674', '-87.640335')
006 OK | 6 ('budget', 'CHIC08', 'Chicago') ('5.11', '41.872643', '-87.62627')
010 OK | 10 ('budget', 'CHIX01', 'Chicago Union Train Station') ('5.71', '41.878674', '-87.640335')
003 ND | 3 ('avis', 'CHIC05', 'Chicago - 18th Street & Wabash Avenue') ('4.05', '41.856873', '-87.62607')
007 OK | 7 ('enterprise', 'E11530', 'Gg Stony Island') ('5.17', '41.727246', '-87.585492')
008 OK | 8 ('enterprise', 'E115DA', 'Gg Gresham') ('5.57', '41.735849', '-87.66861')005 OK |
 5 ('enterprise', 'E115DN', 'Gg South Western') ('4.19', '41.811259', '-87.68454')
011 OK | 11 ('enterprise', 'E115CE', 'Chicago Loop') ('5.82', '41.881861', '-87.633865')
004 OK | 4 ('budget', 'CHIC02', 'Chicago') ('4.05', '41.856873', '-87.62607')
002 OK | 2 ('enterprise', 'E11576', 'Gg South Loop') ('3.17', '41.844252', '-87.623689')
015 OK |

092 OK | 92 ('enterprise', 'E11508', 'Aa Westmont') ('19.16', '41.808409', '-87.9759')
093 OK | 93 ('enterprise', 'E11501', 'Aa Elmhurst') ('19.34', '41.92641', '-87.939792')
094 ND | 94 ('enterprise', 'E115P1', 'Packer Branch Elmhurst') ('19.34', '41.926378', '-87.939894')
095 OK | 95 ('enterprise', 'E11532', 'Gg Highland') ('19.43', '41.538019', '-87.471322')
096 OK | 96 ('enterprise', 'E115GR', 'Gg Griffith') ('19.55', '41.551021', '-87.427419')
097 OK | 97 ('enterprise', 'ORDT61', 'Ohare Intl Arpt') ('19.79', '41.995592', '-87.885729')
099 OK | 99 ('enterprise', 'E115AK', 'Aa W Grand Ave') ('20.15', '41.9305', '-87.954659')
100 OK | 100 ('enterprise', 'E11550', 'Nn-park Ridge') ('20.19', '42.024864', '-87.855535')
101 OK | 101 ('avis', 'CHIW10', 'Lombard') ('20.30', '41.86054', '-87.99008')
098 OK | 98 ('enterprise', 'E115AL', 'Aa Lombard') ('19.86', '41.86029', '-87.98148')
102 OK | 102 ('budget', 'CHIW03', 'Lombard') ('20.30', '41.86054', '-87.99008')
103 OK | 103 ('enterprise', 

183 OK | 183 ('enterprise', 'E11518', 'Aa Joliet') ('33.53', '41.522407', '-88.135687')
184 OK | 184 ('enterprise', 'E115PL', 'Aa Plainfield') ('35.69', '41.54539', '-88.2047')
185 OK | 185 ('avis', 'CHIW09', 'Foxfield Commons Shopping Plaza') ('35.80', '41.921238', '-88.280365')
186 OK | 186 ('budget', 'CHIW01', 'St.charles') ('35.80', '41.921238', '-88.280365')
188 OK | 188 ('enterprise', 'E115AO', 'Aa Oswego') ('36.35', '41.710606', '-88.298573')
187 OK | 187 ('enterprise', 'E115DS', 'Aa Shorewood') ('35.82', '41.523572', '-88.189941')
189 OK | 189 ('enterprise', 'E115N5', 'Nn-mundelein') ('36.51', '42.2392', '-87.9996')
192 OK | 192 ('enterprise', 'E11547', 'Aa East St. Charles') ('36.83', '41.916471', '-88.302333')
191 OK | 191 ('enterprise', 'E11542', 'Aa Aurora') ('36.65', '41.773929', '-88.314159')
190 OK | 190 ('enterprise', 'E11539', 'Gg Valparaiso') ('36.52', '41.46203', '-87.062748')
193 OK | 193 ('avis', 'AUZC01', 'Aurora') ('36.89', '41.786087', '-88.3196')
194 OK | 194 (

In [5]:
addresses = []
for i in range(1,len(results)+1):
    entry = results[i]
    addresses.append(entry['addr']+', '+entry['city']+', '+entry['state']+', '+entry['zip']+', USA')
batchsize = 100
fullbatches = len(addresses)//batchsize
print(len(addresses))
destinations = []
for i in range(0,fullbatches):
    #print(i)
    destinations.append('|'.join(addresses[i*batchsize:(i+1)*batchsize]))
if len(addresses)%batchsize > 0:
    destinations.append('|'.join(addresses[fullbatches*batchsize:]))
print([destinations[i].count('|')+1 for i in range(0,len(destinations))])
origin = '1369 East Hyde Park Blvd, Chicago, IL, 60615, USA'

217
[100, 100, 17]


In [6]:
for i, dest in enumerate(destinations):
    gresponse = getGDistance(origin, dest)
    time.sleep(0.5)
    #print(gresponse)
    if gresponse['status'] == 'OK':        
        elems = gresponse['rows'][0]['elements']
        print('Got gmatrix response for batch {}|{}'.format(i,len(elems)))
        for j,entry in enumerate(elems):
            idx = i*batchsize+j+1
            if (entry['status'] == 'OK'):
                results[idx]['time'] = entry['duration']['value']
                results[idx]['roaddist'] = entry['distance']['value']
                #print(idx,results[idx]['time'],results[idx]['roaddist'])
            else:
                print('{} ENTRY FAILED'.format(idx))
    else:
        print('Gmatrix response for batch {} FAILED'.format(i))

{'destination_addresses': ['5508 S Lake Park Ave, Chicago, IL 60637, USA', '2640 S Michigan Ave, Chicago, IL 60616, USA', '1830 S Wabash Ave, Chicago, IL 60616, USA', '1830 S Wabash Ave, Chicago, IL 60616, USA', '4524 S Western Ave, Chicago, IL 60609, USA', '714 S Wabash Ave, Chicago, IL 60605, USA', '9214 S Stony Island Ave, Chicago, IL 60617, USA', '1820 W 87th St, Chicago, IL 60620, USA', '225 Canal St, Chicago, IL 60606, USA', '225 Canal St, Chicago, IL 60606, USA', '201 W Madison St, Chicago, IL 60606, USA', '318 S Morgan St, Chicago, IL 60607, USA', '555 W Madison St #102, Chicago, IL 60661, USA', '20 E Randolph St, Chicago, IL 60601, USA', 'N Clark St & W Wacker Dr, Chicago, IL 60601, USA', '214 N Clark St, Chicago, IL 60601, USA', '203 N LaSalle Dr, Chicago, IL 60601, USA', '303 W Lake St, Chicago, IL 60606, USA', '4825 S Pulaski Rd, Chicago, IL 60630, USA', '1224 S Western Ave, Chicago, IL 60608, USA', '10 E Grand, Hilton Garage Fl 2, Chicago, IL 60611, United States', '850 N 

Got gmatrix response for batch 0|100
95 ENTRY FAILED
96 ENTRY FAILED
{'destination_addresses': ['1101 E Roosevelt Rd, Lombard, IL 60148, USA', '1101 E Roosevelt Rd, Lombard, IL 60148, USA', '1670 S River Rd, Des Plaines, IL 60018, USA', '501 W N Ave, Villa Park, IL 60181, USA', '9401 N Milwaukee Ave, Niles, IL 60714, USA', '9411 N Greenwood Ave, Niles, IL 60714, USA', '9411 N Greenwood Ave, Niles, IL 60714, USA', '7411 Lemont Rd, Downers Grove, IL 60516, USA', '7411 Lemont Rd, Downers Grove, IL 60516, USA', '4545 Lincoln Hwy, Matteson, IL 60443, USA', '1340 Ogden Ave, Downers Grove, IL 60515, USA', '1340 Ogden Ave, Downers Grove, IL 60515, USA', '1420 Ogden Ave, Downers Grove, IL 60515, USA', '3790 Grant St, Gary, IN 46408, USA', '1233 US-41, Schererville, IN 46375, USA', '21321 Central Ave, Matteson, IL 60443, USA', '19300 La Grange Rd, Mokena, IL 60448, USA', '2420 Landmeier Rd, Elk Grove Village, IL 60007, USA', '2420 Landmeier Rd, Elk Grove Village, IL 60007, USA', '2300 Waukegan R

Got gmatrix response for batch 1|100
104 ENTRY FAILED
114 ENTRY FAILED
115 ENTRY FAILED
125 ENTRY FAILED
133 ENTRY FAILED
134 ENTRY FAILED
137 ENTRY FAILED
138 ENTRY FAILED
149 ENTRY FAILED
152 ENTRY FAILED
159 ENTRY FAILED
160 ENTRY FAILED
178 ENTRY FAILED
185 ENTRY FAILED
186 ENTRY FAILED
187 ENTRY FAILED
188 ENTRY FAILED
189 ENTRY FAILED
190 ENTRY FAILED
192 ENTRY FAILED
197 ENTRY FAILED
198 ENTRY FAILED
{'destination_addresses': ['450 S Milwaukee Ave, Libertyville, IL 60048, USA', '3115 Beacon St, North Chicago, IL 60064, USA', '', '771 Dundee Ave, East Dundee, IL 60118, USA', '', '190 Peterson Road (a.k.a. Buckley Rd), Libertyville, IL 60048, United States', '190 Peterson Rd, Libertyville, IL 60048, USA', '400 S Randall Rd e, Elgin, IL 60123, USA', '26315 US-12, Wauconda, IL 60084, USA', '2801 Washington St, Waukegan, IL 60085, USA', '5000 Spring Hill Ring Rd, West Dundee, IL 60118, USA', '5000 Spring Hill Ring Rd, West Dundee, IL 60118, USA', '34740 Highway 45, Bldg D, Lake Villa

In [9]:
car1 = 'Economy Car'
car2 = 'Compact Car'
car3 = 'Intermediate Car'
car4 = 'Intermediate SUV'
desiredcar = car2
#desiredsort = 'time'
desiredsort = desiredcar
desiredbrand = 'enterprise'

# also, can optimize into some single query...buuuut it works as is
#rvals = np.array([x for x in results.values() if desiredcar in x and desiredsort in x and x['brand'] == desiredbrand])
rvals = np.array([x for x in results.values() if desiredcar in x and desiredsort in x])
orderpreferred = np.array([x[desiredsort] for x in rvals]).argsort()
orderprice = np.array([x[desiredcar] for x in rvals]).argsort()
order = [x for x in orderpreferred if x in orderprice[:70]]

In [10]:
orderedresults = rvals[order]
print(data['pickupDate'],data['pickupTime'],data['dropoffDate'],data['dropoffTime'],desiredcar,car2,car3)
for k in orderedresults:
    #print(k['num'])
    if 'time' in k:
        print('{:03d} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['num'],float(k['dist']),k['time']//60,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))
    else:
        print('{:03d} | {:5.2f} | {:3d} | {:6.2f} | {:6.2f} | {:6.2f} | {:10s} | {}'
              .format(k['num'],float(k['dist']),0,k[desiredcar],k[car2],k[car3],str(k['brand']),k['name']))

11/3/2017 5:00 PM 11/10/2017 5:00 PM Compact Car Compact Car Intermediate Car
128 | 24.53 | 133 | 119.45 | 119.45 | 129.95 | enterprise | Aa Glen Ellyn
109 | 21.31 | 162 | 124.90 | 124.90 | 135.40 | enterprise | Aa South Downers Grove
075 | 17.54 |  93 | 126.12 | 126.12 | 137.21 | enterprise | Nn Skokie
068 | 16.04 |  90 | 126.12 | 126.12 | 137.21 | enterprise | Nn Lincolnwood 53
086 | 18.86 | 103 | 126.12 | 126.12 | 137.21 | enterprise | Morton Grove
088 | 18.96 | 106 | 128.37 | 128.37 | 139.46 | enterprise | Nn North Evanston
215 | 44.16 | 173 | 129.86 | 129.86 | 133.88 | budget     | Gurnee Il Sears
062 | 14.15 |  74 | 129.97 | 129.97 | 140.96 | enterprise | Dd Elmwood Park
050 | 10.97 |  68 | 129.97 | 129.97 | 140.96 | enterprise | Dd Oak Park
067 | 16.04 | 105 | 129.97 | 129.97 | 140.96 | enterprise | Dd Stone Park
198 | 37.18 |   0 | 131.10 | 131.10 | 135.15 | budget     | Lake Zurich Il
207 | 39.56 | 148 | 131.10 | 131.10 | 135.15 | budget     | Libertyville Il
200 | 37.57 | 136

In [ ]:
"""
11/1/2017 11:00 AM 11/8/2017 11:00 AM Compact Car Compact Car Intermediate Car
109 | 21.31 | 162 | 123.61 | 123.61 | 132.69 | enterprise | Aa South Downers Grove
176 | 32.22 | 198 | 126.01 | 126.01 | 135.54 | enterprise | Aa South Naperville
178 | 32.46 |   0 | 126.01 | 126.01 | 135.54 | enterprise | Aa North Plainfield
141 | 27.09 | 191 | 126.02 | 126.02 | 135.56 | enterprise | Aa East Naperville
130 | 24.83 | 187 | 126.02 | 126.02 | 135.56 | enterprise | Aa Lisle
129 | 24.81 | 114 | 126.31 | 126.31 | 135.85 | enterprise | Aa Bolingbrook
166 | 30.85 | 174 | 128.90 | 128.90 | 138.44 | enterprise | Aa Warrenville
175 | 31.67 | 150 | 128.90 | 128.90 | 138.44 | enterprise | Aa East Aurora
161 | 30.14 | 131 | 128.90 | 128.90 | 138.44 | enterprise | Aa West Ogden
170 | 31.04 | 125 | 128.90 | 128.90 | 138.44 | enterprise | Aa Naperville North
059 | 13.64 | 106 | 130.61 | 130.61 | 140.20 | enterprise | Aa Lagrange
158 | 29.21 | 135 | 131.10 | 131.10 | 135.15 | budget     | Highland Park
136 | 26.20 | 140 | 132.39 | 132.39 | 141.84 | enterprise | Aa Lockport
085 | 18.57 | 116 | 132.39 | 132.39 | 141.84 | enterprise | Aa Willowbrook
177 | 32.26 | 143 | 133.65 | 133.65 | 143.19 | enterprise | Aa Crest Hill
113 | 21.41 | 168 | 133.65 | 133.65 | 143.19 | enterprise | Aa North Downers Grove
104 | 20.90 |   0 | 133.65 | 133.65 | 143.19 | enterprise | Aa Villa Park
098 | 19.86 | 125 | 133.65 | 133.65 | 143.19 | enterprise | Aa Lombard
093 | 19.34 | 128 | 133.65 | 133.65 | 143.19 | enterprise | Aa Elmhurst
092 | 19.16 | 102 | 133.65 | 133.65 | 143.19 | enterprise | Aa Westmont
144 | 27.68 | 141 | 133.65 | 133.65 | 143.19 | enterprise | Aa Carol Stream
099 | 20.15 | 142 | 133.65 | 133.65 | 143.19 | enterprise | Aa W Grand Ave
050 | 10.97 |  68 | 134.97 | 134.97 | 144.96 | enterprise | Dd Oak Park
067 | 16.04 | 105 | 134.98 | 134.98 | 136.97 | enterprise | Dd Stone Park
062 | 14.15 |  74 | 134.98 | 134.98 | 136.97 | enterprise | Dd Elmwood Park
068 | 16.04 |  90 | 136.20 | 136.20 | 138.20 | enterprise | Nn Lincolnwood 53
075 | 17.54 |  93 | 136.20 | 136.20 | 138.20 | enterprise | Nn Skokie
086 | 18.86 | 103 | 136.20 | 136.20 | 138.20 | enterprise | Morton Grove
088 | 18.96 | 106 | 138.45 | 138.45 | 140.45 | enterprise | Nn North Evanston
132 | 25.02 | 130 | 138.52 | 138.52 | 142.80 | budget     | Northbrook
162 | 30.52 | 123 | 141.22 | 141.22 | 151.30 | enterprise | Aa Hanover Park
157 | 29.21 | 135 | 141.25 | 141.25 | 143.84 | avis       | Highland Park Ford
146 | 27.93 | 182 | 141.84 | 141.84 | 151.29 | enterprise | Nn-hyatt-deerfield
077 | 17.76 |  88 | 142.95 | 142.95 | 144.96 | enterprise | Nn South Evanston N2
150 | 28.39 | 122 | 143.52 | 143.52 | 153.06 | enterprise | Nn-highland Park
066 | 15.24 |  81 | 149.89 | 149.89 | 152.06 | enterprise | Nn Norwood Park C1
120 | 22.80 | 127 | 149.95 | 149.95 | 159.94 | enterprise | Nn-glenview
181 | 32.84 | 177 | 151.27 | 151.27 | 161.35 | enterprise | Aa Palatine
135 | 26.13 | 137 | 151.30 | 151.30 | 161.38 | enterprise | Nn-wheeling
167 | 30.91 | 159 | 151.30 | 151.30 | 161.38 | enterprise | Nn-napleton Dge Jeep Chr
087 | 18.96 | 132 | 151.31 | 151.31 | 154.14 | enterprise | Aa Holiday Inn
128 | 24.53 | 133 | 156.75 | 156.75 | 166.20 | enterprise | Aa Glen Ellyn
061 | 13.81 |  81 | 158.44 | 158.44 | 160.61 | enterprise | Napleton Chrysler
054 | 11.85 |  72 | 158.44 | 158.44 | 160.61 | enterprise | Nn Riverview
076 | 17.63 |  97 | 162.33 | 162.33 | 178.45 | enterprise | Schiller Park
121 | 23.02 | 110 | 170.19 | 170.19 | 179.64 | enterprise | Gg Steger
138 | 26.83 |   0 | 170.19 | 170.19 | 179.64 | enterprise | Gg New Lenox
101 | 20.30 | 126 | 170.52 | 170.52 | 177.72 | avis       | Lombard
064 | 14.69 |  51 | 170.67 | 170.67 | 180.66 | enterprise | Gg Hammond
114 | 21.60 |   0 | 170.68 | 170.68 | 180.66 | enterprise | Gg Gary
033 |  8.08 |  47 | 170.70 | 170.70 | 191.70 | enterprise | Euro Collission
127 | 24.47 | 182 | 171.22 | 171.22 | 174.67 | budget     | The Hyatt Of Lisle Il
112 | 21.35 | 169 | 171.22 | 171.22 | 174.67 | budget     | Downers Grove Il
139 | 27.01 | 191 | 171.22 | 171.22 | 174.67 | budget     | Naperville
172 | 31.24 | 137 | 171.22 | 171.22 | 174.67 | budget     | Aurora Il Sears
169 | 30.94 | 134 | 171.22 | 171.22 | 174.67 | budget     | Naperville Il
117 | 22.13 | 112 | 171.81 | 171.81 | 181.35 | enterprise | Gg Frankfort And Mokena
123 | 23.70 | 174 | 172.14 | 172.14 | 179.42 | avis       | Addison
049 | 10.82 |  68 | 173.02 | 173.02 | 168.01 | budget     | Oak Park
154 | 28.59 | 183 | 173.07 | 173.07 | 175.46 | budget     | Romeoville, Il Lewis Airport
"""